##Imports

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.layers import Dense, LSTM, GlobalAveragePooling1D, GRU, Embedding, TextVectorization, Input, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

##Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##Dataset and Pre-processing

In [29]:
#Load the dataset
ds_path = "/content/drive/MyDrive/Colab Notebooks/Resume Recommendation/final_data.csv"
df = pd.read_csv(ds_path)
df=df.dropna()

#Combine the train features
df['Combined'] = df["Skills"] + " " + df["Objective"] + " " + df["Experience"] + " "+ df["Education"]

#Train-Test split
x = df["Combined"]
y = df['Desired Industry']
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.33, random_state=42)

#Encode the labels
coder=LabelEncoder()
classes=y_train.unique()
y_train = coder.fit_transform(y_train)

#Categocarical converstion
# y_train=tf.keras.utils.to_categorical(
#     y_train, num_classes=13
# )

##Tokenization and Embedding

In [30]:
max_vocab_len=10000
max_length=5000

#Tokenization Layer
tokenizer = TextVectorization(max_tokens=max_vocab_len,
                             pad_to_max_tokens=True,
                             output_sequence_length=max_length) 
tokenizer.adapt(x_train)

#Embedding layer
embed=Embedding(input_dim=max_vocab_len,
          output_dim=128,
          input_length=max_length)

##Model

In [ ]:
#Model
inputs = Input(shape=(1,), dtype=tf.string)
num_input=tokenizer(inputs)
num_input=embed(num_input)
num_input=GRU(128, return_sequences=True)(num_input)
num_input=LSTM(64, return_sequences=True)(num_input)
num_input=GRU(32)(num_input)
outputs = Dense(1, activation="softmax")(num_input)

model_1=tf.keras.Model(inputs, outputs, name="model_1")
model_1.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=["accuracy"])
model_1.summary()

In [32]:
model_1_history = model_1.fit(x_train, y_train, epochs=10)